<a href="https://colab.research.google.com/github/hariharanrl/NLP/blob/master/NLP_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount Google Drive

In [0]:
import pandas as pd
import os
os.chdir('/content/drive/My Drive')

#Spacy (Not needed)

In [0]:
#import spacy
#nlp=spacy.load('en_core_web_sm')#lg large library #sm small library

In [0]:
#stopwords=nlp.Defaults.stop_words

In [0]:
#len(stopwords)

326

In [0]:
#nlp.vocab['@user'].is_stop

False

In [0]:
#to add a stop word
#nlp.Defaults.stop_words.add('@user') 
#nlp.vocab['@user'].is_stop=True

#Download the Vader Class from NLTK

In [2]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

##Load the data

In [0]:
data=pd.read_csv('task_b_distant.tsv',sep='\t')

##Install and Import Tweet Preprocessor

In [4]:
!pip install tweet-preprocessor

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7947 sha256=f7934a49d18fd7dae200f22704adb43ec90990888990c2a5e3f84633d79c24c5
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor


In [0]:
import preprocessor as p

##Clean the tweet in the imported data using preprocessor

In [6]:
p.clean(data.loc[0]['text'])

'His ass need to stay up'

In [7]:
data['cleaned_tweet']=data['text'].apply(lambda text:p.clean(text))
data.head()

,id,text,average,std,cleaned_tweet
0,1159533703758061570,@USER His ass need to stay up 😂😂,0.176201,0.267800,His ass need to stay up
1,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.173800,0.239891,Trump is a fucking idiot his dementia is getti...
2,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.199833,0.212598,HELL YES! His grinned and thumbs up are disgus...
3,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.144666,0.235204,Can't wait to see the shit show his death will...
4,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.116019,0.186851,This guys is dumb check his latest tweets he i...


##Use the vader class to calculate the polarity scores for the cleaned tweets

In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [9]:
sid.polarity_scores(data.loc[3]['text'])

{'compound': 0.0258, 'neg': 0.218, 'neu': 0.559, 'pos': 0.223}

##Label the tweet using the compound score

In [10]:
data['scores']=data['cleaned_tweet'].apply(lambda text:sid.polarity_scores(text))
data['label']=data['scores'].apply(lambda d:d['compound'])
data['class']=data['label'].apply(lambda c:'off' if c<=-0.05 else 'not')
data.head()

,id,text,average,std,cleaned_tweet,scores,label,class
0,1159533703758061570,@USER His ass need to stay up 😂😂,0.176201,0.267800,His ass need to stay up,"{'neg': 0.412, 'neu': 0.588, 'pos': 0.0, 'comp...",-0.5423,off
1,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.173800,0.239891,Trump is a fucking idiot his dementia is getti...,"{'neg': 0.489, 'neu': 0.511, 'pos': 0.0, 'comp...",-0.7713,off
2,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.199833,0.212598,HELL YES! His grinned and thumbs up are disgus...,"{'neg': 0.469, 'neu': 0.252, 'pos': 0.279, 'co...",-0.6988,off
3,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.144666,0.235204,Can't wait to see the shit show his death will...,"{'neg': 0.231, 'neu': 0.533, 'pos': 0.237, 'co...",0.0258,not
4,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.116019,0.186851,This guys is dumb check his latest tweets he i...,"{'neg': 0.423, 'neu': 0.577, 'pos': 0.0, 'comp...",-0.7650,off


##Check for any Null texts

In [0]:
data.isnull().sum()

id               0
text             0
average          0
std              0
cleaned_tweet    0
scores           0
label            0
class            0
dtype: int64

##Check for blanks

In [0]:
blanks=[]
for i,j,text,avg,std in data.itertuples():
  if type(text)==str:
    if text.isspace():
      blanks.append(i)
print(len(blanks))

In [11]:
data['class'].value_counts()

off    160693
not     28034
Name: class, dtype: int64

#Training and test split 

In [0]:
from sklearn.model_selection import train_test_split
X=data['cleaned_tweet']
y=data['class']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

##Building a Naive bayes and Linear SVM model 

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

#text_cls_nb=Pipeline([('tfidf',TfidfVectorizer(stop_words='english')),('clf',MultinomialNB()),])
text_cls_svm=Pipeline([('tfidf',TfidfVectorizer(min_df=5,max_df=0.9,ngram_range=(1,5))),('clf',LinearSVC()),])


In [37]:
tfidf=TfidfVectorizer(stop_words='english',min_df=5,max_df=0.9,ngram_range=(1,1))
tfidf.fit_transform(X_train)


<150981x11023 sparse matrix of type '<class 'numpy.float64'>'
	with 943893 stored elements in Compressed Sparse Row format>

##Fit the model with the train data

In [58]:
#text_cls_nb.fit(X_train,y_train)
text_cls_svm.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.9, max_features=None,
                                 min_df=5, ngram_range=(1, 5), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

##Prediction, Confusion Matrix and Classification Report

In [0]:
#prediction=text_cls_nb.predict(X_test)
prediction=text_cls_svm.predict(X_test)

In [60]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,prediction))

[[ 3424  2136]
 [  962 31224]]


In [61]:
print(metrics.classification_report(y_test,prediction))

              precision    recall  f1-score   support

         not       0.78      0.62      0.69      5560
         off       0.94      0.97      0.95     32186

    accuracy                           0.92     37746
   macro avg       0.86      0.79      0.82     37746
weighted avg       0.91      0.92      0.91     37746



In [62]:
print(metrics.accuracy_score(y_test,prediction))

0.9179250781539766


In [68]:
print(metrics.f1_score(y_test,prediction,average='weighted'))

0.9138161483480721


In [69]:
print(text_cls_svm.predict(['A great day to begin!!! Good morning']))

['not']


In [64]:
print(text_cls_svm.predict(['Bull Shit']))

['off']
